<a href="https://colab.research.google.com/github/leonardo3422/big-data/blob/https%2Fcantina-senac.netlify.app/C%C3%B3pia_de_Tributo_e_Competencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Equipe
Mariana duarte

Renan amendola

Filipe aymar

Gabriel Castelo

Leonardo André

Helenne Duarte

## Contexto
Este conjunto de dados contém as informações referentes à receita tributária auferida por diferentes recolhimentos que tenham características econômicas de tributo, nas diversas esferas de arrecadação e que resultem em um fluxo de recursos financeiros direcionados da sociedade para o Estado.

Com base nessas informações, é possível realizar uma variedade de análises que oferecem insights valiosos para diferentes áreas de interesse:

-- Evolução da Arrecadação Tributária ao Longo dos Anos: Ao analisar os dados ao longo do período de 2002 a 2021, é possível identificar tendências de crescimento, quedas ou estabilidade na arrecadação tributária em diferentes esferas de governo e tipos de tributos. Isso permite entender melhor a dinâmica da receita tributária ao longo do tempo e as influências econômicas e políticas que moldaram essas tendências.

-- Comparação entre Diferentes Tipos de Tributos e Esferas de Arrecadação: É possível comparar a arrecadação de tributos federais, estaduais e municipais, bem como analisar a contribuição relativa de cada tipo de tributo para a receita total. Isso pode fornecer insights sobre a distribuição da carga tributária entre diferentes níveis de governo e setores da economia.

-- Análise de Eficiência Fiscal: Ao examinar o percentual da receita tributária em relação ao Produto Interno Bruto (PIB) em cada ano, é possível avaliar a eficiência da arrecadação tributária em relação ao tamanho da economia. Isso pode ajudar a identificar áreas onde melhorias na eficiência fiscal são necessárias e avaliar o impacto das políticas tributárias na atividade econômica.

-- Avaliações do Impacto Econômico da Arrecadação Tributária: Ao estudar a relação entre a arrecadação tributária e indicadores econômicos, como crescimento do PIB, emprego e investimento, é possível avaliar o impacto da tributação na economia como um todo. Isso pode fornecer insights importantes para a formulação de políticas econômicas e fiscais.

Em resumo, o conjunto de dados "Tributo e Competência" oferece uma riqueza de informações que podem ser exploradas de várias maneiras para entender melhor a dinâmica da arrecadação tributária, avaliar a eficiência fiscal e analisar o impacto da tributação na economia. Essas análises podem ser úteis para formuladores de políticas públicas, pesquisadores, economistas e outros profissionais interessados em questões fiscais e econômicas.

## Exploração

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=9aec9a7b26c5e6447d3dc4f59ca7bdf35853d43a4e1e5d7d1e78fb65e1061683
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Tributo_Competencia") \
    .getOrCreate()

In [ ]:
import gdown
import os

url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTBcBCB4vGSOwPz3tF3SOQw4quk9vggzHvG4mWk4dhvLkXFB-0dWvvINQ4qslNB3jYarnUdCG5ARrTu/pub?output=csv'
output = 'data/tributocompetencia.csv'

os.makedirs(os.path.dirname(output), exist_ok=True)

gdown.download(url, output, quiet=False)


/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://docs.google.com/spreadsheets/d/e/2PACX-1vTBcBCB4vGSOwPz3tF3SOQw4quk9vggzHvG4mWk4dhvLkXFB-0dWvvINQ4qslNB3jYarnUdCG5ARrTu/pub?output=csv
To: /content/data/tributocompetencia.csv
102kB [00:00, 49.3MB/s]


'data/tributocompetencia.csv'

In [ ]:
# Lendo o arquivo CSV com PySpark
df = spark.read.csv(output, header=True, inferSchema=True)

# Mostrando as primeiras linhas do DataFrame para verificação
df.show(5)

+--------------+--------------------+----------------+--------------------+---------------------------+-----------------+
|Ano-calendário|         Competência|       Orçamento|           Descrição|Valor da Receita Tributária|Percentual do PIB|
+--------------+--------------------+----------------+--------------------+---------------------------+-----------------+
|          2002|Total dos Tributo...|           União|Total da Receita ...|                477725,6437|      0,320882411|
|          2002|Tributos do Gover...|Orçamento Fiscal|Tributos do Gover...|                330496,8209|       0,22199063|
|          2002|Tributos do Gover...|Orçamento Fiscal|    Orçamento Fiscal|                117337,9936|       0,07881448|
|          2002|Tributos do Gover...|Orçamento Fiscal|Imposto de Renda ...|                83528,69112|       0,05610519|
|          2002|Tributos do Gover...|Orçamento Fiscal|Imposto de Renda ...|                 4078,31919|      0,002739357|
+--------------+--------

In [ ]:
# Contando o número de registros
num_rows = df.count()
print(f"Quantidade de registros no dataset: {num_rows}")


Quantidade de registros no dataset: 920


Formatação dos valores

Agrupando por Ano-calendário e somando a Receita Tributária

In [ ]:
from pyspark.sql import functions as F

# Agrupando por Ano-calendário e somando a Receita Tributária
df.groupBy("Ano-calendário") \
  .agg(F.sum("Valor da Receita Tributária").alias("Total Receita Tributária")) \
  .orderBy("Ano-calendário") \
  .show()


+--------------+------------------------+
|Ano-calendário|Total Receita Tributária|
+--------------+------------------------+
|          2002|                    NULL|
|          2003|                    NULL|
|          2004|                    NULL|
|          2005|                    NULL|
|          2006|                    NULL|
|          2007|                    NULL|
|          2008|                    NULL|
|          2009|                    NULL|
|          2010|                    NULL|
|          2011|                    NULL|
|          2012|                    NULL|
|          2013|                    NULL|
|          2014|                    NULL|
|          2015|                    NULL|
|          2016|                    NULL|
|          2017|                     0.0|
|          2018|                    NULL|
|          2019|                    NULL|
|          2020|                    NULL|
|          2021|                    NULL|
+--------------+------------------

Filtrando por Tributos do Governo Estadual e ordenando por Valor da Receita Tributária

In [ ]:
# Filtrando por Tributos do Governo Estadual
df.filter(df['Orçamento'] == 'Governo Estadual') \
  .orderBy(df['Valor da Receita Tributária'].desc()) \
  .show()


+--------------+--------------------+----------------+--------------------+---------------------------+-----------------+
|Ano-calendário|         Competência|       Orçamento|           Descrição|Valor da Receita Tributária|Percentual do PIB|
+--------------+--------------------+----------------+--------------------+---------------------------+-----------------+
|          2006|Tributos do Gover...|Governo Estadual|                ITCD|                968,9700489|      0,000402154|
|          2005|Tributos do Gover...|Governo Estadual|Outros Tributos E...|                8901,803984|      0,004101109|
|          2004|Tributos do Gover...|Governo Estadual|                IPVA|                8829,081123|      0,004509808|
|          2003|Tributos do Gover...|Governo Estadual|                ITCD|                873,1107554|      0,000508228|
|          2019|Tributos do Gover...|Governo Estadual|                ITCD|                 8582,40824|      0,001161491|
|          2020|Tributos

Agrupando por Tributo e Ano-calendário

In [ ]:
from pyspark.sql import functions as F

# Agrupando por Tributo e Ano-calendário e somando a Receita Tributária
contribuicao_tributos = df.groupBy("Descrição", "Ano-calendário") \
                          .agg(F.sum("Valor da Receita Tributária").alias("Total Receita Tributária")) \
                          .orderBy("Descrição", "Ano-calendário")

# Exibindo o resultado
contribuicao_tributos.show(truncate=False)


+-----------------+--------------+------------------------+
|Descrição        |Ano-calendário|Total Receita Tributária|
+-----------------+--------------+------------------------+
|Cide Combustíveis|2002          |NULL                    |
|Cide Combustíveis|2003          |NULL                    |
|Cide Combustíveis|2004          |NULL                    |
|Cide Combustíveis|2005          |NULL                    |
|Cide Combustíveis|2006          |NULL                    |
|Cide Combustíveis|2007          |NULL                    |
|Cide Combustíveis|2008          |NULL                    |
|Cide Combustíveis|2009          |NULL                    |
|Cide Combustíveis|2010          |NULL                    |
|Cide Combustíveis|2011          |NULL                    |
|Cide Combustíveis|2012          |NULL                    |
|Cide Combustíveis|2013          |NULL                    |
|Cide Combustíveis|2014          |NULL                    |
|Cide Combustíveis|2015          |NULL  

In [ ]:
# Excluindo linhas com valores nulos ou NaN
df_sem_null_nan = df.dropna()

Conversão de Tipos de Dados:

colunas para tipos de dados apropriados, por exemplo, converter colunas numéricas de string para float para facilitar operações matemáticas.

In [ ]:
from pyspark.sql.types import FloatType

df = df.withColumn("Valor_Receita_Tributaria", df["Valor_Receita_Tributaria"].cast(FloatType()))
df = df.withColumn("Percentual do PIB", df["Percentual do PIB"].cast(FloatType()))


Agrupamento e Agregação:

dados por ano, competência ou outra coluna relevante e realizar agregações (como soma, média, máximo, mínimo) para obter insights agregados.

In [ ]:
df_agregado = df.groupBy("Ano-calendário").sum("Valor da Receita Tributária")


Manipulação de Strings:

Limpar ou modificar strings em colunas, por exemplo, remover espaços extras ou caracteres especiais, ou extrair informações específicas.

In [ ]:
from pyspark.sql.functions import trim

df = df.withColumn("Descrição", trim(df["Descrição"]))


Filtragem de Dados:

Filtrar dados com base em condições específicas para focar em subsets relevantes do dataset.
Exemplo de comando para filtrar apenas tributos do governo federal:

In [ ]:
df_federal = df.filter(df["Competência"] == "Tributos do Governo Federal")


Ordenação de Dados:

Ordenar os dados com base em uma ou mais colunas para facilitar a análise ou apresentação dos resultados.
Exemplo de comando para ordenar por valor da receita tributária em ordem decrescente:

In [ ]:
df_ordenado = df.orderBy(df["Valor da Receita Tributária"].desc())


Renomeação de Colunas:

Renomear colunas para nomes mais descritivos ou simplificados.
Exemplo de comando para renomear uma coluna:

In [ ]:
df = df.withColumnRenamed("Valor da Receita Tributária", "Valor_Receita_Tributaria")


Definição de uma função para formatação de reais

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def formatar_reais(valor):
    if valor is None:
        return None
    return f"R${valor:,.2f}"

formatar_reais_udf = udf(formatar_reais, StringType())

df = df.withColumn("Valor_Receita_Tributaria_Formatado", formatar_reais_udf("Valor_Receita_Tributaria"))


Visualização geral

In [ ]:
# Contando o número total de registros no DataFrame
num_rows = df.count()

# Exibindo o total de registros
print(f"Total de registros no dataset: {num_rows}")

# Exibindo a estrutura dos dados
print("Estrutura dos dados:")
df.printSchema()


Total de registros no dataset: 920
Estrutura dos dados:
root
 |-- Ano-calendário: integer (nullable = true)
 |-- Competência: string (nullable = true)
 |-- Orçamento: string (nullable = true)
 |-- Descrição: string (nullable = true)
 |-- Valor_Receita_Tributaria: float (nullable = true)
 |-- Percentual do PIB: float (nullable = true)
 |-- Receita Formatada: string (nullable = true)
 |-- Valor_Receita_Tributaria_Formatado: string (nullable = true)

